In [1]:
!pip install -q transformers accelerate bitsandbytes pillow gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 28.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.2 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
langchain-core 0.3.79 requires packaging<26.0.0,>=23.2.0, but you have packaging 26.0rc2 which is incompatible.
fastai 2.8.4 requires fastcore<1.9,>=1.8.0, but you have fastcore 1.11.3 which is incompatible.


In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from transformers import BitsAndBytesConfig
from PIL import Image
import torchvision.transforms as T
import gradio as gr
import gc


2026-02-17 15:04:33.160499: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771340673.399410      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771340673.466628      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771340674.041887      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771340674.041943      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771340674.041948      55 computation_placer.cc:177] computation placer alr

In [4]:
model_name = "Qwen/Qwen2.5-VL-7B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(model_name)

print("Model loaded successfully!")


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

Model loaded successfully!


In [5]:
prompt = """
You are a professional jewelry authentication expert.

Your task is to compare two jewelry images and output ONLY a final similarity score (0–100).

CRITICAL RULES:
- Ignore lighting, shadows, background, hand presence, scale ruler, brightness, camera quality, and image resolution.
- Mentally isolate the jewelry from the background before comparison.
- Catalogue images may contain multiple items or multiple views.
- If multiple items exist, compare ONLY the pair that shares the dominant matching design.
- Do NOT average across different pieces.

--------------------------------------------------
STEP 1 — JEWELRY TYPE CHECK (MANDATORY FIRST)
--------------------------------------------------

Identify the primary jewelry type in each image.

If the primary jewelry types are clearly different:
→ Immediately assign a score between 0 and 20.
→ STOP comparison.

If types match, continue.

--------------------------------------------------
STEP 2 — OUTER SILHOUETTE GATE
--------------------------------------------------

If outer silhouette clearly differs:
→ Final score must be below 40.

If silhouette matches:
→ Continue.
Do NOT later penalize for the same structural difference again.

--------------------------------------------------
STEP 3 — CORE STRUCTURE
--------------------------------------------------

If core structure clearly differs:
→ Subtract 25.

Never apply more than ONE major structural penalty.

--------------------------------------------------
STEP 4 — MATERIAL / METAL CHECK
--------------------------------------------------

If metal color is clearly different:
→ Subtract 25.

If one is two-tone and other single-tone:
→ Subtract 15.

Material difference should NOT reduce score below 50
if structure is otherwise identical.

--------------------------------------------------
STEP 5 — DETAILED FEATURE SCORING
--------------------------------------------------

Start from 100.

Subtract ONLY for clearly visible mismatches:

- 20 if center stone shape differs
- 15 if stone count differs
- 10 for symmetry/layout differences
- 10 for proportion differences
- 5–10 for small visible variation

--------------------------------------------------
STABILITY RULES
--------------------------------------------------

If type, silhouette, and structure match:
→ Score must NOT fall below 60 unless material differs.

If same design + same metal:
→ Score should be 85–100.

--------------------------------------------------
OUTPUT RULE
--------------------------------------------------

Return ONLY the final similarity score number (0–100).
Do not explain.
Only output the number.
"""


In [18]:
def compare_images(image1, image2):

    torch.cuda.empty_cache()

    # Resize SAME as your notebook (1024)
    transform = T.Resize((1024, 1024))
    image1 = transform(image1.convert("RGB"))
    image2 = transform(image2.convert("RGB"))

    with torch.no_grad():

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "image"},
                    {"type": "text", "text": prompt}
                ]
            }
        ]

        text = processor.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        inputs = processor(
            text=[text],
            images=[image1, image2],
            return_tensors="pt"
        ).to(model.device)

        # 🔥 Memory-optimized generation
        output = model.generate(
            **inputs,
            max_new_tokens=80,      # reduced from 300 (huge memory saver)
            temperature=0.2,
            do_sample=False,
            use_cache=False         # critical for VRAM stability
        )

        response = processor.decode(
            output[0],
            skip_special_tokens=True
        )

    torch.cuda.empty_cache()
    gc.collect()

    return response.strip()


In [22]:
import gradio as gr

# Your existing backend function
# def compare_images(img1, img2):
#     ...
#     return similarity_score

demo = gr.Interface(
    fn=compare_images,
    inputs=[
        gr.Image(type="pil", label="Upload Jewelry Image 1"),
        gr.Image(type="pil", label="Upload Jewelry Image 2"),
    ],
    outputs=gr.Textbox(label="Similarity Score"),
    title="AI Jewelry Authentication System (7B VLM)",
    description="Quantized 7B Vision-Language Model running on Kaggle GPU"
)

# For latest Gradio (4.x+)
demo.queue()
demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://be78fb331061e7f63d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
